# Projekt "Credit Risk Modeling"

In diesem Kapitel wird ein praxisnahes Data-Science-Projekt behandelt, bei dem ein Kreditrisikomodell entwickelt wird. Ziel des Projekts ist es, das Ausfallrisiko von Kreditanträgen zu bewerten. Dazu wird ein Datensatz genutzt, der Informationen über Kreditnehmer enthält, beispielsweise Alter, Beruf, Kontostand oder Verwendungszweck des Kredits.
<br>
<br>
Die Zielvariable ist das Merkmal `Risk`, das zwischen "good" (geringes Risiko) und "bad" (hohes Risiko) unterscheidet.
<br>
<br>
Unser Datensatz `german_credit_data.csv`, enthält insgesamt über 1.000 Zeilen von Kreditnehmern. Jeder Datensatz beschreibt einen einzelnen Kreditantrag anhand verschiedener Merkmale:
- Age – Alter der Antragsteller (in Jahren)
- Sex – Geschlecht (männlich oder weiblich)
- Job – Berufskategorie (0 = niedrigste Qualifikation, 3 = höchste Qualifikation)
- Housing – Wohnsituation (eigene Wohnung, Miete oder kostenloses Wohnen)
- Saving accounts – Kategorie des Sparguthabens (z.B. "little", "moderate", "rich")
- Checking account – Kategorie des Girokontos (z.B. "little", "moderate", "rich")
- Credit amount – Höhe des aufgenommenen Kredits
- Duration – Laufzeit des Kredits in Monaten
- Purpose – Zweck des Kredits (z.B. Auto, Radio/TV, Urlaub, Ausbildung, Geschäft)
- Risk – Zielvariable: Kreditrisiko
  - good = Kunde gilt als risikoarm
  - bad = Kunde gilt als risikoreich

## 1. Daten erkunden

In [48]:
import pandas as pd

In [49]:
df = pd.read_csv("german_credit_data.csv")
df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,996,40,male,3,own,little,little,3857,30,car,good
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad


Betrachten wir einige grundlägende statistische Informationen zu dem Datensatz:

In [50]:
df.describe()

,Unnamed: 0,Age,Job,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,35.546000,1.904000,3271.258000,20.903000
std,288.819436,11.375469,0.653614,2822.736876,12.058814
min,0.000000,19.000000,0.000000,250.000000,4.000000
25%,249.750000,27.000000,2.000000,1365.500000,12.000000
50%,499.500000,33.000000,2.000000,2319.500000,18.000000
75%,749.250000,42.000000,2.000000,3972.250000,24.000000
max,999.000000,75.000000,3.000000,18424.000000,72.000000


Aus diesen Informationen lassen sich folgende Beobachtungen ableiten:

**1. Age:**
- Minimum: 19 Jahre, Maximum: 75 Jahre
- Durchschnitt: ca. 36 Jahre
- Die meisten Kreditnehmer sind also im jungen bis mittleren Erwachsenenalter.

**2. Job:**
- Mittelwert liegt bei 1,9, Median ebenfalls bei 2
- Das zeigt, dass die Mehrheit der Kreditnehmer eine mittlere Berufskategorie (2) hat.

**3. Kredithöhe (Credit amount):**
- Werte zwischen 250 € und 18.424 €
- Durchschnitt: ca. 3.271 €
- 25 % der Kredite liegen unter 1.365 €, während 75 % unter 3.972 € bleiben.
- Die meisten Kredite sind relativ klein, es gibt aber einige sehr hohe Kredite (Ausreißer).

**4. Laufzeit (Duration in Monaten):**
- Minimum: 4 Monate, Maximum: 72 Monate (6 Jahre)
- Durchschnitt: ca. 21 Monate, Median: 18 Monate
- Die meisten Kredite haben also eine Laufzeit von 1 bis 2 Jahren.

**5. Indexspalte (Unnamed: 0):**
- Reine Zählspalte (0–999), hat keinen inhaltlichen Mehrwert und sollte später entfernt werden.

Im nächsten Schritt wollen wir uns anschauen, wie oft die Zielvariable `Risk` in unserem Datensatz vorkommt. Diese Spalte gibt an, ob ein Kredit als "good" (geringes Risiko) oder "bad" (hohes Risiko) eingestuft wurde. Das ist wichtig, weil wir dadurch erkennen können, ob die Daten ausgeglichen sind (gleich viele "good" und "bad") oder ob eine Seite deutlich überwiegt.
<br>
<br>
Ein solches Ungleichgewicht nennt man Klassenungleichgewicht. Das kann später beim Trainieren eines Machine-Learning-Modells eine große Rolle spielen, weil ein Modell sonst dazu neigt, fast immer die häufigere Klasse vorherzusagen.

In [51]:
print(df["Risk"].value_counts())

Risk
good    700
bad     300
Name: count, dtype: int64


Wir erhalten also folgende Verteilung:
- 700 Kredite sind als "good" (geringes Risiko) eingestuft.
- 300 Kredite sind als "bad" (hohes Risiko) eingestuft.

Das bedeutet rund 70% der Daten gehören zur Klasse "good", und nur 30% zur Klasse "bad". Deshalb muss man bei der Modellierung darauf achten, mit Methoden wie gewichteten Modellen, Balancing-Techniken oder geeigneten Algorithmen zu arbeiten, die auch mit unausgeglichenen Daten umgehen können. Darauf werden wir noch zurückkommen.
<br>
<br>
Verschaffen wir uns noch einen Überblick über den Aufbau des Datensatzes:

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   int64 
 1   Age               1000 non-null   int64 
 2   Sex               1000 non-null   object
 3   Job               1000 non-null   int64 
 4   Housing           1000 non-null   object
 5   Saving accounts   817 non-null    object
 6   Checking account  606 non-null    object
 7   Credit amount     1000 non-null   int64 
 8   Duration          1000 non-null   int64 
 9   Purpose           1000 non-null   object
 10  Risk              1000 non-null   object
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


Aus diesen Informationen lassen sich weitere wichtige Punkte ableiten:

**1. Größe des Datensatzes:**
- Es gibt 1.000 Zeilen (Einträge) und 11 Spalten (Merkmale).

**2. Datentypen:**
- 5 Spalten sind numerisch (int64: z.B. Alter, Kreditbetrag).
- 6 Spalten sind kategorial (object: z.B. Geschlecht, Wohnsituation).
- Das ist wichtig, weil numerische und kategoriale Daten später unterschiedlich verarbeitet werden müssen.

**3. Vollständigkeit der Daten:**
- Die meisten Spalten sind vollständig gefüllt (1000 Werte).
- Zwei Spalten haben jedoch fehlende Werte:
- Saving accounts: nur 817 Werte (183 fehlen)
- Checking account: nur 606 Werte (394 fehlen)
- Diese Lücken müssen wir bei der Datenaufbereitung berücksichtigen (z.B. durch Entfernen oder Ersetzen).

**4. Technische Spalte:**
- `Unnamed: 0` ist nur ein Index (0–999) und liefert keinen inhaltlichen Mehrwert. Diese Spalte kann man später entfernen.

Nachdem wir uns einen Überblick über den Datensatz verschafft haben, wollen wir nun genauer in eine einzelne Spalte schauen: `Job`. Wir lassen uns alle verschiedenen Ausprägungen (Kategorien) anzeigen, die in dieser Spalte vorkommen:

In [53]:
df["Job"].unique()

array([2, 1, 3, 0])

n der Spalte `Job` gibt es genau vier verschiedene Kategorien, die durch die Zahlen 0–3 dargestellt werden. Diese Zahlen sind keine Messwerte, sondern Codierungen für Job-Kategorien. Sie beschreiben also unterschiedliche berufliche Situationen:
- `0`: unqualifiziert / arbeitslos
- `1`: unqualifizierte Tätigkeit
- `2`: qualifizierte Tätigkeit
- `3`: hochqualifizierte Tätigkeit oder Management

Im nächsten Schritt prüfen wir unsere Daten, ob und wo im Datensatz Werte fehlen:

In [54]:
df.isna().sum()

Unnamed: 0            0
Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

Besonders auffällig sind zwei Spalten:
- `Saving accounts` 183 fehlende Werte
- `Checking account` 394 fehlende Werte

Fehlende Werte können ein Problem für Machine Learning Modelle sein, da die meisten Algorithmen nur mit vollständigen Daten arbeiten können.
<br>
<br>
Nachdem wir uns die fehlenden Werte angesehen haben, wollen wir nun prüfen, ob es im Datensatz doppelte Zeilen gibt:

In [55]:
df.duplicated().sum()

np.int64(0)

Es sind keine Dupplikate enthalten. Der Datensatz ist in diesem Punkt sauber und kann direkt weiterverarbeitet werden. Wir haben gesehen, dass in den Spalten `Saving accounts` und `Checking account` viele Werte fehlen. Solche fehlenden Werte (NaN) können Probleme machen, wenn wir später ein Machine-Learning-Modell trainieren.
<br>
<br>
An dieser Stelle halten wir es einfach und entfernen wir alle Zeilen, die mindestens einen fehlenden Wert enthalten. Dadurch bleiben nur noch vollständig ausgefüllte Datensätze übrig:

In [56]:
df = df.dropna().reset_index(drop=True)

Wir überprüfen noch einmal, ob nun wirklich keine fehlenden Werte im Datensatz sind:

In [57]:
df.isna().sum()

Unnamed: 0          0
Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
Risk                0
dtype: int64

In unserem Datensatz gibt es die Spalte `Unnamed: 0`. Diese Spalte enthält nur fortlaufende Zahlen von 0 bis 999 und ist eigentlich nichts anderes als ein technischer Index. Da wir für die Analyse und Modellierung bereits den Pandas-Index verwenden, bringt diese Spalte keinen zusätzlichen Informationswert. Wir entfernen sie:

In [58]:
df.drop(columns="Unnamed: 0", inplace=True)

Nun sollte keine Spalte mit der Bezeichnung `Unnamed: 0` vorhanden sein:

In [59]:
df.columns

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose', 'Risk'],
      dtype='object')

Nachdem wir den Datensatz bereinigt haben, wollen wir uns die Verteilung der numerischen Merkmale anschauen.
Dafür zeichnen wir Histogramme:

In [60]:
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1) Numerische Spalten ermitteln:
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# 2) Eigene Bins je Merkmal:
bins_cfg = {
    "Age":            dict(start=15, end=80,    size=5),  
    "Credit amount":  dict(start=0,  end=20000, size=1000), 
    "Duration":       dict(start=0,  end=80,    size=6),
    "Job":            dict(start=-0.5, end=3.5, size=1),
}

# 3) Subplot-Grid abhängig von der Anzahl der numerischen Spalten:
cols_per_row = 2
rows = math.ceil(len(numeric_cols) / cols_per_row)

fig1 = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=numeric_cols,
    horizontal_spacing=0.08,
    vertical_spacing=0.15
)

# 4) Histogramme hinzufügen:
for i, col in enumerate(numeric_cols):
    r = i // cols_per_row + 1
    c = i % cols_per_row + 1
    fig1.add_trace(
        go.Histogram(
            x=df[col],
            xbins=bins_cfg.get(col, None),
            marker_line_color="black",
            marker_line_width=1,
            name=col
        ),
        row=r, col=c
    )

# 5) Optik anpassen:
fig1.update_xaxes(showgrid=False)
fig1.update_yaxes(showgrid=False)
fig1.update_layout(
    title="Verteilung der numerischen Merkmale",
    showlegend=False,
    bargap=0.05,
    height=350 * rows 
)

fig1

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Age',
              'type': 'histogram',
              'x': {'bdata': ('Fi01IxwZGBY8HCAsLDAsJCoiPyQbHi' ... '0eLxYXKBYdJDlAGTEcGhkhHTAeKBcb'),
                    'dtype': 'i1'},
              'xaxis': 'x',
              'xbins': {'end': 80, 'size': 5, 'start': 15},
              'yaxis': 'y'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Job',
              'type': 'histogram',
              'x': {'bdata': ('AgICAwMCAgIBAgEDAgECAQICAgICAg' ... 'ECAgEBAQIAAgECAgIDAgICAwEDAwIC'),
                    'dtype': 'i1'},
              'xaxis': 'x2',
              'xbins': {'end': 3.5, 'size': 1, 'start': -0.5},
              'yaxis': 'y2'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Credit amount',
              'type': 'histogram',
              'x': {'bdata': ('PxfKHgYTJBtyFA8F1BAfBq8EewUCBS' ... 'TCIGsLJSCZBZEYsxnPBncPEQ81B+AR'),
                    'dtype': 'i2'},
              'xaxis': 'x3',
              'xbins': {'end': 20000, 'size': 1000, 'start': 0},
              'yaxis': 'y3'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Duration',
              'type': 'histogram',
              'x': {'bdata': ('MCoYJB4MMAwYDxgYBgoMBgwHPBIYEg' ... 'wYFQYbCw8YPBgGDx4VJA8qGBgkHi0t'),
                    'dtype': 'i1'},
              'xaxis': 'x4',
              'xbins': {'end': 80, 'size': 6, 'start': 0},
              'yaxis': 'y4'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Age',
                                'x': 0.23,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Job',
                                'x': 0.77,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Credit amount',
                                'x': 0.23,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.425,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Duration',
                                'x': 0.77,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.425,
                                'yanchor': 'bottom',
                                'yref': 'paper'}],
               'bargap': 0.05,
               'height': 700,
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Verteilung der numerischen Merkmale'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 0.46], 'showgrid': False},
               'xaxis2': {'anchor': 'y2', 'domain': [0.54, 1.0], 'showgrid': False},
               'xaxis3': {'anchor': 'y3', 'domain': [0.0, 0.46], 'showgrid': False},
               'xaxis4': {'anchor': 'y4', 'domain': [0.54, 1.0], 'showgrid': False},
               'yaxis': {'anchor': 'x', 'domain': [0.575, 1.0], 'showgrid': False},
    

Aus den Histogrammen können wir einige wichtige Informationen entnehmen:

**1. Alter (Age):**
- Die meisten Kreditnehmer sind zwischen 20 und 40 Jahren alt.
- Danach nimmt die Häufigkeit deutlich ab.
- Sehr alte Kreditnehmer (über 70) sind selten.
- Das zeigt: Der Datensatz spiegelt vor allem jüngere bis mittelalte Erwachsene wider.
- Die Verteilung ist rechtsschief: Viele jüngere Menschen, immer weniger mit zunehmendem Alter. Das ist typisch für Kreditnehmer, da vor allem jüngere Erwachsene Kredite aufnehmen.

**2. Job:**
- Die häufigste Kategorie ist "2" (qualifizierte Tätigkeit).
- Kategorien "0" (arbeitslos/unqualifiziert) und "3" (hochqualifiziert) sind deutlich seltener.
- Das heißt, die Mehrheit der Kreditnehmer befindet sich in einem mittleren Beschäftigungsniveau.
- Nicht gleichmäßige Verteilung: Kategorie 2 dominiert stark, während 0 und 3 selten sind. Der Schwerpunkt liegt deutlich auf mittleren Jobs.

**3.Kredithöhe (Credit amount):**
- Die Verteilung ist stark rechtsschief:
- Viele Kredite liegen im Bereich unter 5.000 €.
- Es gibt nur wenige sehr hohe Kredite (bis über 15.000 €).
- Typisch: Die meisten Kunden beantragen relativ kleine Kredite, nur wenige beantragen sehr hohe Summen.
- Sehr stark rechtsschief: Die meisten Kredite sind klein (unter 5.000 €), nur wenige sehr groß. Die Ausreißer nach oben ziehen den Wertebereich stark auseinander.

**4. Kreditlaufzeit (Duration):**
- Am häufigsten sind Laufzeiten von 12 bis 24 Monaten.
- Wenige Kredite haben sehr lange Laufzeiten (über 60 Monate).
- Das zeigt: Kredite sind überwiegend auf kurze bis mittlere Zeiträume ausgelegt.
- Ebenfalls rechtsschief: Kürzere Laufzeiten (bis 24 Monate) dominieren. Lange Laufzeiten sind selten und bilden Ausreißer.

Nachdem wir die Verteilungen der numerischen Merkmale mit Histogrammen betrachtet haben, wollen wir nun auch Boxplots erstellen.
Ein Boxplot zeigt uns auf einen Blick:
- Median (mittlerer Wert),
- Unteres und oberes Quartil (die "Box" = 50% der Werte),
- Spannweite der Daten (die Whisker),
- Sowie Ausreißer (Werte, die weit außerhalb der restlichen Daten liegen).

In [61]:

# 1) Subplot-Raster festlegen:
cols_per_row = 2
rows = math.ceil(len(numeric_cols) / cols_per_row)

fig2 = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=numeric_cols,
    horizontal_spacing=0.08,
    vertical_spacing=0.15
)

# 2) Boxplots je numerischer Spalte hinzufügen:
for i, col in enumerate(numeric_cols):
    r = i // cols_per_row + 1
    c = i % cols_per_row + 1
    fig2.add_trace(
        go.Box(
            y=df[col],
            name=col,
            boxpoints="outliers",   
            marker_line_color="black",
            marker_line_width=1,
            whiskerwidth=0.8
        ),
        row=r, col=c
    )

# 3) Layout anpassen:
fig2.update_xaxes(showgrid=False)
fig2.update_yaxes(showgrid=True)
fig2.update_layout(
    title="Boxplots der numerischen Merkmale",
    showlegend=False,
    height=300 * rows
)

fig2


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'boxpoints': 'outliers',
              'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Age',
              'type': 'box',
              'whiskerwidth': 0.8,
              'xaxis': 'x',
              'y': {'bdata': ('Fi01IxwZGBY8HCAsLDAsJCoiPyQbHi' ... '0eLxYXKBYdJDlAGTEcGhkhHTAeKBcb'),
                    'dtype': 'i1'},
              'yaxis': 'y'},
             {'boxpoints': 'outliers',
              'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Job',
              'type': 'box',
              'whiskerwidth': 0.8,
              'xaxis': 'x2',
              'y': {'bdata': ('AgICAwMCAgIBAgEDAgECAQICAgICAg' ... 'ECAgEBAQIAAgECAgIDAgICAwEDAwIC'),
                    'dtype': 'i1'},
              'yaxis': 'y2'},
             {'boxpoints': 'outliers',
              'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Credit amount',
              'type': 'box',
              'whiskerwidth': 0.8,
              'xaxis': 'x3',
              'y': {'bdata': ('PxfKHgYTJBtyFA8F1BAfBq8EewUCBS' ... 'TCIGsLJSCZBZEYsxnPBncPEQ81B+AR'),
                    'dtype': 'i2'},
              'yaxis': 'y3'},
             {'boxpoints': 'outliers',
              'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Duration',
              'type': 'box',
              'whiskerwidth': 0.8,
              'xaxis': 'x4',
              'y': {'bdata': ('MCoYJB4MMAwYDxgYBgoMBgwHPBIYEg' ... 'wYFQYbCw8YPBgGDx4VJA8qGBgkHi0t'),
                    'dtype': 'i1'},
              'yaxis': 'y4'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Age',
                                'x': 0.23,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Job',
                                'x': 0.77,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Credit amount',
                                'x': 0.23,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.425,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Duration',
                                'x': 0.77,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.425,
                                'yanchor': 'bottom',
                                'yref': 'paper'}],
               'height': 600,
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Boxplots der numerischen Merkmale'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 0.46], 'showgrid': False},
               'xaxis2': {'anchor': 'y2', 'domain': [0.54, 1.0], 'showgrid': False},
               'xaxis3': {'anchor': 'y3', 'domain': [0.0, 0.46], 'showgrid': False},
               'xaxis4': {'anchor': 'y4', 'domain': [0.54, 1.0], 'showgrid': False},
               'yaxis': {'anchor': 'x', 'domain': [0.575, 1.0], 'showgrid': True},
             

Die Boxplots bestätigen die Ergebnisse aus den Histogrammen:
- Kredithöhe und Laufzeit haben viele Ausreißer nach oben.
- Alter ist überwiegend zwischen 20–40 Jahren konzentriert.
- Job ist kategorial, daher ist der Boxplot hier nur eingeschränkt aussagekräftig.

Wichtig: Ausreißer sind keine Fehler, aber man muss entscheiden, ob man sie mitmodelliert oder behandelt (z.B. Transformation, Skalierung oder Entfernen).
<br>
<br>
Mit den Boxplots haben wir gesehen, dass es einige Ausreißer bei der Kreditlaufzeit (Duration) gibt, die über 70 Monate liegen.
Jetzt wollen wir uns diese Fälle gezielt anschauen:

In [62]:
df.query("Duration > 70")

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
358,24,male,2,own,moderate,moderate,5595,72,radio/TV,bad


Es ist nur ein starker Ausreißer über dem Wert 70 vorhanden. Einige Kredite laufen länger als 60 Monate (also über 5 Jahre), laut dem Boxplot sind sie ebenfalls Ausreißer. Schauen wir uns an wie viele es sind:

In [63]:
df.query("Duration >= 60")

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
18,63,male,2,own,little,little,6836,60,business,bad
176,24,female,3,own,moderate,moderate,7408,60,car,bad
199,60,female,3,free,moderate,moderate,14782,60,vacation/others,bad
358,24,male,2,own,moderate,moderate,5595,72,radio/TV,bad
378,27,male,3,own,little,moderate,14027,60,car,bad
489,42,male,2,free,little,moderate,6288,60,education,bad
507,36,male,2,rent,little,little,7297,60,business,bad


Die Abfrage hat 7 Fälle ergeben, bei denen die Kreditlaufzeit mindestens 60 Monate beträgt. Auffällig sind dabei mehrere Punkte:

**1. Höhe der Kredite:**
- Alle Kredite in dieser Gruppe sind relativ hoch (zwischen ca. 5.500 € und fast 15.000 €).
- Längere Laufzeiten sind oft mit höheren Kreditbeträgen verbunden.

**2. Verwendungszweck:**
- Die Zwecke sind vielfältig: Business, Car, Vacation/others, Radio/TV, Education.
- Lange Laufzeiten kommen also nicht nur bei einem Zweck vor, sondern in mehreren Bereichen.

**3. Risikoeinschätzung:**
- Alle diese Kredite sind mit "bad" (hohes Risiko) klassifiziert.
- Offensichtlich betrachtet die Bank Kredite mit langen Laufzeiten eher als riskant.

**4. Altersverteilung:**
- Sowohl jüngere Kreditnehmer (24–27 Jahre) als auch ältere (60+ Jahre) kommen hier vor.
- Es gibt also kein einheitliches Muster beim Alter – aber die Risikoeinstufung ist in allen Fällen gleich ("bad).

Nachdem wir uns die numerischen Merkmale mit Histogrammen und Boxplots angesehen haben, wollen wir nun auch die kategorialen Merkmale untersuchen. Bei kategorialen Daten interessiert uns vor allem:
- Welche Kategorien gibt es überhaupt?
- Wie häufig kommen die einzelnen Kategorien vor?

Um das sichtbar zu machen, erstellen wir für jede kategoriale Spalte ein Balkendiagramm, das die Verteilung zeigt:

In [64]:
# 1) Kategorische Spalten ermitteln:
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()

# 2) Subplot-Raster anlegen:
cols_per_row = 3
rows = math.ceil(len(categorical_cols) / cols_per_row)

fig3 = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=categorical_cols,
    horizontal_spacing=0.08,
    vertical_spacing=0.15
)

# 3) Für jede kategoriale Spalte ein Balkendiagramm hinzufügen:
for i, col in enumerate(categorical_cols):
    r = i // cols_per_row + 1
    c = i % cols_per_row + 1
    
    value_counts = df[col].value_counts().sort_values(ascending=False)
    
    fig3.add_trace(
        go.Bar(
            x=value_counts.index.astype(str), 
            y=value_counts.values,             
            marker_line_color="black",
            marker_line_width=1,
            name=col
        ),
        row=r, col=c
    )

# 4) Layout anpassen:
fig3.update_xaxes(showgrid=False, tickangle=45)
fig3.update_yaxes(showgrid=False)
fig3.update_layout(
    title="Verteilung der kategorialen Merkmale",
    showlegend=False,
    height=350 * rows
)

fig3

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Sex',
              'type': 'bar',
              'x': array(['male', 'female'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'YgGoAA==', 'dtype': 'i2'},
              'yaxis': 'y'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Housing',
              'type': 'bar',
              'x': array(['own', 'rent', 'free'], dtype=object),
              'xaxis': 'x2',
              'y': {'bdata': 'XQFsAEEA', 'dtype': 'i2'},
              'yaxis': 'y2'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Saving accounts',
              'type': 'bar',
              'x': array(['little', 'moderate', 'quite rich', 'rich'], dtype=object),
              'xaxis': 'x3',
              'y': {'bdata': 'nAFAABcAFwA=', 'dtype': 'i2'},
              'yaxis': 'y3'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Checking account',
              'type': 'bar',
              'x': array(['little', 'moderate', 'rich'], dtype=object),
              'xaxis': 'x4',
              'y': {'bdata': '9QDgADUA', 'dtype': 'i2'},
              'yaxis': 'y4'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Purpose',
              'type': 'bar',
              'x': array(['car', 'radio/TV', 'furniture/equipment', 'business', 'education',
                          'repairs', 'vacation/others', 'domestic appliances'], dtype=object),
              'xaxis': 'x5',
              'y': {'bdata': 'rQCEAGsANQAcAA4ACQAGAA==', 'dtype': 'i2'},
              'yaxis': 'y5'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Risk',
              'type': 'bar',
              'x': array(['good', 'bad'], dtype=object),
              'xaxis': 'x6',
              'y': {'bdata': 'IwHnAA==', 'dtype': 'i2'},
              'yaxis': 'y6'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Sex',
                                'x': 0.13999999999999999,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Housing',
                                'x': 0.49999999999999994,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Saving accounts',
                                'x': 0.86,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Checking account',
                                'x': 0.13999999999999999,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.425,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                

Fassen wir einige wichtige Beobachtungen zusammen:

**1. Geschlecht (Sex):**
- Es gibt deutlich mehr männliche Kreditnehmer als weibliche.
- Männer machen ungefähr 2/3 der Daten aus.

**2. Wohnsituation (Housing):**
- Die Mehrheit der Kreditnehmer wohnt im eigenen Haus.
- Mieten ist weniger verbreitet, und nur wenige wohnen "kostenlos" (z.B. bei Eltern).

**3.Sparguthaben (Saving accounts):**
- Sehr viele Kreditnehmer haben nur wenig Ersparnisse ("little").
- Kategorien wie "moderate", "quite rich" und "rich" sind deutlich seltener.
- Wenig Ersparnisse deuten darauf hin, dass viele Kunden finanziell nicht stark abgesichert sind

**4. Kontostand (Checking account):**
- Auch hier dominieren die Kategorien "little" und "moderate".
- "Rich" ist deutlich seltener.
- Viele Kunden verfügen also nur über geringe bis mittlere Kontostände.

**5. Verwendungszweck (Purpose):**
- Am häufigsten wird ein Kredit für den Autokauf ("car") aufgenommen.
- Danach folgen Radio/TV und Möbel.
- Bildung, Urlaub oder Reparaturen sind seltener vertreten.
- Kredite werden oft für größere Anschaffungen im Alltag genutzt.

**6. Kreditrisiko (Risk):**
- Es gibt mehr "good"-Kredite (niedriges Risiko) als "bad"-Kredite (hohes Risiko).
- Das Verhältnis ist ungefähr 70% "good" zu 30% "bad".
- Der Datensatz ist leicht unausgeglichen, aber beide Klassen sind vorhanden.

Nachdem wir die einzelnen Merkmale bereits einzeln betrachtet haben, wollen wir nun untersuchen, wie stark die numerischen Variablen miteinander zusammenhängen. Dazu berechnen wir die Korrelationsmatrix. Korrelation beschreibt, ob zwei Variablen einen gemeinsamen Trend haben:
- Werte nahe +1: Starker positiver Zusammenhang (beide steigen gemeinsam).
- Werte nahe -1: Starker negativer Zusammenhang (eine steigt, während die andere fällt).
- Werte um 0: Kein oder nur sehr schwacher Zusammenhang.

In [65]:
corr = df[numeric_cols].corr()
corr

,Age,Job,Credit amount,Duration
Age,1.000000,0.039771,0.082014,0.001549
Job,0.039771,1.000000,0.334721,0.200794
Credit amount,0.082014,0.334721,1.000000,0.613298
Duration,0.001549,0.200794,0.613298,1.000000


Die Korrelationsmatrix in Tabellenform ist oft schwer zu überblicken.
Darum wollen wir die Werte grafisch als Heatmap darstellen:

In [66]:
import plotly.express as px

corr = df[numeric_cols].corr().round(2)

fig4 = px.imshow(
    corr,
    text_auto=True,       
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,   
    aspect="auto",
    title="Korrelationsmatrix der numerischen Merkmale"
)

fig4.update_layout(margin=dict(l=60, r=20, t=60, b=40))
fig4

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: %{z}<extra></extra>',
              'name': '0',
              'texttemplate': '%{z}',
              'type': 'heatmap',
              'x': array(['Age', 'Job', 'Credit amount', 'Duration'], dtype=object),
              'xaxis': 'x',
              'y': array(['Age', 'Job', 'Credit amount', 'Duration'], dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAAAA8D97FK5H4XqkP3sUrkfher' ... 'mZmZnJP4XrUbgeheM/AAAAAAAA8D8='),
                    'dtype': 'f8',
                    'shape': '4, 4'}}],
    'layout': {'coloraxis': {'cmax': 1,
                             'cmin': -1,
                             'colorscale': [[0.0, 'rgb(5,48,97)'], [0.1,
                                            'rgb(33,102,172)'], [0.2,
                                            'rgb(67,147,195)'], [0.3,
                                            'rgb(146,197,222)'], [0.4,
                                            'rgb(209,229,240)'], [0.5,
                                            'rgb(247,247,247)'], [0.6,
                                            'rgb(253,219,199)'], [0.7,
                                            'rgb(244,165,130)'], [0.8,
                                            'rgb(214,96,77)'], [0.9,
                                            'rgb(178,24,43)'], [1.0,
                                            'rgb(103,0,31)']]},
               'margin': {'b': 40, 'l': 60, 'r': 20, 't': 60},
               'template': '...',
               'title': {'text': 'Korrelationsmatrix der numerischen Merkmale'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0]},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0]}}
})

Bisher haben wir die Merkmale getrennt voneinander betrachtet. Jetzt wollen wir herausfinden, ob es einen Zusammenhang zwischen der Job-Kategorie und der Höhe des aufgenommenen Kredits gibt:

In [67]:
df.groupby("Job")["Credit amount"].mean()

Job
0    1767.857143
1    2250.715517
2    3129.130990
3    5648.784810
Name: Credit amount, dtype: float64

Die Analyse zeigt einen klaren Zusammenhang zwischen Job-Kategorie und Kredithöhe. Menschen in höheren Job-Kategorien nehmen im Durchschnitt deutlich größere Kredite auf als Personen in niedrigeren Kategorien.
<br>
<br>
Wir wollen nun untersuchen, ob es Unterschiede in der durchschnittlichen Kredithöhe zwischen Männern und Frauen gibt:

In [68]:
df.groupby("Sex")["Credit amount"].mean()

Sex
female    2937.202381
male      3440.833333
Name: Credit amount, dtype: float64

Männer nehmen in diesem Datensatz im Durchschnitt höhere Kredite auf als Frauen. Der Unterschied ist zwar sichtbar, aber nicht so groß, dass er allein entscheidend wäre.
<br>
<br>
Bisher haben wir uns Durchschnittswerte nur für eine Gruppierung angeschaut (z. B. nach Job oder nach Geschlecht).
Jetzt wollen wir gleichzeitig zwei Merkmale kombinieren und sehen, wie sich die Kredithöhen danach unterscheiden:

In [69]:
pd.pivot_table(df, values="Credit amount", index="Housing", columns="Purpose")

Purpose,business,car,domestic appliances,education,furniture/equipment,radio/TV,repairs,vacation/others
Housing,,,,,,,,
free,4705.000000,5180.314286,NaN,5314.250000,4419.444444,2097.000000,1190.0,7842.666667
own,3725.973684,3120.485437,1333.5,2625.076923,3031.100000,2307.613861,2993.5,10321.833333
rent,6180.833333,3398.285714,NaN,2627.857143,2890.285714,2138.000000,2384.0,NaN


Hier sind einige wichtige Beobachtungen aus der Pivot-Tabelle:

**1. Wohnsituation macht einen Unterschied:**

Mieter (rent) haben für business im Schnitt die höchsten Beträge (~6.181 €).
- Eigentümer `own` liegen häufig mittig (z.B. business ~3.726 €, radio/TV ~2.308 €).
- Kostenlos wohnend `free` ist teils hoch (z.B. car ~5.180 €, education ~5.314 €).

**2. Zweckabhängige Ausreißer:**
- `vacation/others` ist bei Eigentümern besonders hoch (~10.322 €) und auch bei free erhöht (~7.843 €).
- `car` ist bei free deutlich teurer (~5.180 €) als bei own/rent (~3.120–3.398 €).
`radio/TV` ist in allen Wohnformen relativ niedrig (~2.100–2.300 €).
- `repairs` und `domestic appliances` sind eher niedrig (bei `domestic appliances` sogar nur Werte für own vorhanden: ~1.334 €).

**3. Datenlücken (NaN) beachten:**
- NaN bedeutet: Für diese Kombination liegen keine Fälle vor (z.B. domestic appliances bei free/rent, vacation/others bei rent).
- Interpretationen sollten deshalb vorsichtig sein – fehlende Kombinationen können das Gesamtbild verzerren.

Die Art der Wohnsituation beeinflusst, wie viel Geld Menschen für bestimmte Zwecke aufnehmen. Große Kredite sind besonders mit Business, Urlaub und Auto verbunden, während Konsumgüter wie Radio/TV nur geringe Kreditsummen benötigen.
<br>
<br>
Jetzt stellen wir zwei wichtige Merkmale, Alter und Kreditbetrag, in einem gemeinsamen Diagramm dar. Zusätzlich binden wir weitere Informationen ein:
- Geschlecht (über die Farbe),
- Kreditlaufzeit (über die Größe der Punkte).

So entsteht eine multidimensionale Visualisierung, in der wir mehrere Zusammenhänge gleichzeitig erkennen können:

In [70]:
fig5 = px.scatter(
    df,
    x="Age",
    y="Credit amount",
    color="Sex",       
    size="Duration",   
    opacity=0.7,         
    title="Alter vs. Kreditbetrag",
    labels={"Age": "Alter", "Credit amount": "Kreditbetrag"}
)

fig5.update_layout(
    legend_title="Geschlecht",
    margin=dict(l=40, r=40, t=60, b=40)
)

fig5

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('Sex=female<br>Alter=%{x}<br>Kr' ... '=%{marker.size}<extra></extra>'),
              'legendgroup': 'female',
              'marker': {'color': '#636efa',
                         'opacity': 0.7,
                         'size': {'bdata': ('MAwwDA8YGAwMMAYSJCoYEg8hFQwMJA' ... 'gSDAsMFRgwCRIJDAkPMBgMGBgYBhUP'),
                                  'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.18,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'female',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': ('FhkYFhwgLCohOhcXFyk6GxcXGR0sHi' ... 'MbGCg1HBkgFhUWFxwuHhwdOR05QBwZ'),
                    'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('PxcPBdQQHwZ7BQIFIzGZAcIF/xdIBa' ... 'XpAjUHpiAREGUHzwSpBOoE8QJrC5kF'),
                    'dtype': 'i2'},
              'yaxis': 'y'},
             {'hovertemplate': ('Sex=male<br>Alter=%{x}<br>Kred' ... '=%{marker.size}<extra></extra>'),
              'legendgroup': 'male',
              'marker': {'color': '#EF553B',
                         'opacity': 0.7,
                         'size': {'bdata': ('KhgkHhgGCgwGBzwSGBItEgoJDBIeGy' ... 'wYEiQJHBgVBhsLDzwPHiQqGBgkHi0t'),
                                  'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.18,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'male',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': ('LTUjHDwsMCwkIj8kGx4ZJSUYGiwYHj' ... 'oeKBglLR4vFhcoFiQZMRohHTAeKBcb'),
                    'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('yh4GEyQbchSvBFcKwQgMB14Fbwm0Gn' ... 'WBHPAEwiAlIJEYsxnPBncPEQ81B+AR'),
                    'dtype': 'i2'},
              'yaxis': 'y'}],
    'layout': {'legend': {'itemsizing': 'constant', 'title': {'text': 'Geschlecht'}, 'tracegroupgap': 0},
               'margin': {'b': 40, 'l': 40, 'r': 40, 't': 60},
               'template': '...',
               'title': {'text': 'Alter vs. Kreditbetrag'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Alter'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Kreditbetrag'}}}
})

Im Diagramm sieht man, dass die meisten Kredite im Bereich zwischen 1.000 € und 5.000 € aufgenommen werden, unabhängig vom Alter. Besonders viele Punkte liegen bei jüngeren Personen im Alter zwischen 20 und 40 Jahren.
<br>
<br>
Es gibt allerdings auch einige Ausreißer mit sehr hohen Kreditbeträgen (bis über 15.000 €), die bei unterschiedlichen Altersgruppen vorkommen. Die größeren Kreise zeigen zudem, dass hohe Kredite oft mit längeren Laufzeiten verbunden sind.
<br>
<br>
Zwischen Männern und Frauen erkennt man keinen extrem deutlichen Unterschied – beide Geschlechter nehmen Kredite in vergleichbaren Größenordnungen auf. Allerdings scheinen Männer im Datensatz häufiger vertreten zu sein.
<br>
<br>
Die Mehrheit der Kredite bewegt sich im niedrigen bis mittleren Bereich, junge Kreditnehmer sind stärker vertreten, und hohe Kredite gehen meist mit langen Laufzeiten einher.
<br>
<br>
Wir wollen uns nun anschauen, wie sich der Kreditbetrag in Abhängigkeit vom Sparguthaben (Saving accounts) verteilt. Dafür eignet sich ein Violinplot:
- Er kombiniert die Vorteile von Boxplots (Median, Quartile, Ausreißer) mit einer Dichtedarstellung.
- So kann man sofort erkennen, wo die meisten Werte liegen und ob es Bereiche gibt, in denen besonders viele oder wenige Kreditbeträge vorkommen.

Damit können wir untersuchen, ob Kunden mit höherem Sparguthaben auch höhere Kredite aufnehmen – oder ob sich der Unterschied eher gering ausfällt:

In [71]:
fig6 = px.violin(
    df,
    x="Saving accounts",
    y="Credit amount",
    box=True,      
    points="all",        
    color="Saving accounts",
    title="Verteilung der Kreditbeträge nach Sparguthaben",
    labels={"Saving accounts": "Sparguthaben", "Credit amount": "Kreditbetrag"}
)

fig6.update_layout(showlegend=False, margin=dict(l=40, r=40, t=60, b=40))
fig6


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'box': {'visible': True},
              'hovertemplate': 'Sparguthaben=%{x}<br>Kreditbetrag=%{y}<extra></extra>',
              'legendgroup': 'little',
              'marker': {'color': '#636efa'},
              'name': 'little',
              'offsetgroup': 'little',
              'orientation': 'v',
              'points': 'all',
              'scalegroup': 'True',
              'showlegend': True,
              'type': 'violin',
              'x': array(['little', 'little', 'little', ..., 'little', 'little', 'little'],
                         shape=(412,), dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': {'bdata': ('PxfKHgYTJBtyFA8F1BAfBq8EewUjMc' ... 'Ec8QLCICUgmQWRGLMZzwZ3DxEPNQc='),
                    'dtype': 'i2'},
              'y0': ' ',
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'box': {'visible': True},
              'hovertemplate': 'Sparguthaben=%{x}<br>Kreditbetrag=%{y}<extra></extra>',
              'legendgroup': 'moderate',
              'marker': {'color': '#EF553B'},
              'name': 'moderate',
              'offsetgroup': 'moderate',
              'orientation': 'v',
              'points': 'all',
              'scalegroup': 'True',
              'showlegend': True,
              'type': 'violin',
              'x': array(['moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate', 'moderate', 'moderate',
                          'moderate', 'moderate', 'moderate', 'moderate'], dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': {'bdata': ('AgUMB+oWKxhEMckIPgqpBe4NNxsIBb' ... 'UJSQNEA2YFNwl/B/0L6gXwBGsL4BE='),
                    'dtype': 'i2'},
              'y0': ' ',
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'box': {'visible': True},
              'hovertemplate': 'Sparguthaben=%{x}<br>Kreditbetrag=%{y}<extra></extra>',
              'legendgroup': 'quite rich',
              'marker': {'color': '#00cc96'},
              'name': 'quite rich',
              'offsetgroup': 'quite rich',
              'orientation': 'v',
              'points': 'all',
              'scalegroup': 'True',
              'showlegend': True,
              'type': 'violin',
              'x': array(['quite rich', 'quite rich', 'quite rich', 'quite rich', 'quite rich',
                          'quite rich', 'quite rich', 'quite rich', 'quite rich', 'quite rich',
                          'quite rich', 'quite rich', 'quite rich', 'quite rich', 'quite rich',
                          'quite rich', 'quite rich', 'quite rich', 'quite rich', 'quite rich',
                          'quite rich', 'quite rich', 'quite rich'], dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': {'bdata': 'VwqGBEgFggWQBrkQPgn+AlIBjQyuDhYJqwM4CQwLFgOzCcgFpiAVDREQ2AnqBA==', 'dtype': 'i2'},
              'y0': ' ',
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'box': {'visible': True},
              'hovertempla

In der Grafik sieht man die Verteilung der Kreditbeträge in Abhängigkeit vom Sparguthaben. Auffällig ist, dass in allen Kategorien die meisten Kredite im Bereich zwischen etwa 1.000 € und 5.000 € liegen. Besonders viele Personen haben nur geringe Ersparnisse (`little`), und gerade in dieser Gruppe kommen auch sehr hohe Kredite vor – bis hin zur Obergrenze im Datensatz. Das zeigt, dass auch Kunden mit wenig Rücklagen Kredite in großer Höhe aufnehmen.
<br>
<br>
Bei den Kategorien `moderate`, `quite rich` und `rich` gibt es zwar insgesamt weniger Fälle, die Muster sind aber ähnlich: Die meisten Kredite liegen im unteren bis mittleren Bereich, und es tauchen immer wieder einzelne Ausreißer nach oben auf. Daraus können wir schließen, dass die Höhe der Ersparnisse nicht allein bestimmt, wie hoch ein Kredit ist. In allen Gruppen finden sich sowohl kleine als auch große Kreditbeträge, sodass andere Faktoren – zum Beispiel Beruf oder Einkommen – wahrscheinlich wichtiger sind.
<br>
<br>
Bisher haben wir die Verteilung der Kredithöhen, Altersgruppen und anderer Merkmale betrachtet. Nun wollen wir uns das Zielmerkmal Risk genauer anschauen. Als erstes lassen wir uns anzeigen, wie hoch der prozentuale Anteil der beiden Kategorien good (geringes Risiko) und bad (hohes Risiko) im Datensatz ist:

In [72]:
print(df["Risk"].value_counts(normalize=True) * 100)

Risk
good    55.747126
bad     44.252874
Name: proportion, dtype: float64


Die Auswertung zeigt, dass etwa 56 % der Kredite als `good` (niedriges Risiko) und rund 44 % als `bad` (hohes Risiko) eingestuft sind. Damit ist der Datensatz zwar nicht perfekt ausgeglichen, aber beide Klassen sind ausreichend stark vertreten. Für die Modellierung heißt das: Wir müssen uns bewusst machen, dass es ein kleines Ungleichgewicht gibt, aber die Verteilung ist noch so ausgeglichen, dass man beide Klassen gut vergleichen und mit ihnen arbeiten kann.
<br>
<br>
Wir haben bereits gesehen, dass das Merkmal Risk unser Ziel darstellt und zwischen good (geringes Risiko) und bad (hohes Risiko) unterscheidet. Nun wollen wir untersuchen, ob sich bestimmte numerische Merkmale wie Alter, Kredithöhe oder Laufzeit zwischen diesen beiden Gruppen unterscheiden. Dafür erstellen wir Boxplots, die die Verteilungen der Werte getrennt nach good und bad darstellen:

In [73]:
cols = ["Age", "Credit amount", "Duration"]

# Subplot-Raster (1 Reihe, 3 Spalten):
fig7 = make_subplots(
    rows=1, cols=3,
    subplot_titles=cols
)

# Für jede Spalte einen Boxplot hinzufügen:
for i, col in enumerate(cols):
    fig7.add_trace(
        go.Box(
            x=df["Risk"],
            y=df[col],
            boxpoints="outliers",   
            marker=dict(color="lightblue"),
            line=dict(color="black"),
            name=col
        ),
        row=1, col=i+1
    )

fig7.update_layout(
    title="Boxplots der Merkmale nach Risiko-Klasse",
    showlegend=False,
    margin=dict(l=40, r=40, t=60, b=40)
)

fig7


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'boxpoints': 'outliers',
              'line': {'color': 'black'},
              'marker': {'color': 'lightblue'},
              'name': 'Age',
              'type': 'box',
              'x': array(['bad', 'good', 'bad', ..., 'good', 'bad', 'good'],
                         shape=(522,), dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('Fi01IxwZGBY8HCAsLDAsJCoiPyQbHi' ... '0eLxYXKBYdJDlAGTEcGhkhHTAeKBcb'),
                    'dtype': 'i1'},
              'yaxis': 'y'},
             {'boxpoints': 'outliers',
              'line': {'color': 'black'},
              'marker': {'color': 'lightblue'},
              'name': 'Credit amount',
              'type': 'box',
              'x': array(['bad', 'good', 'bad', ..., 'good', 'bad', 'good'],
                         shape=(522,), dtype=object),
              'xaxis': 'x2',
              'y': {'bdata': ('PxfKHgYTJBtyFA8F1BAfBq8EewUCBS' ... 'TCIGsLJSCZBZEYsxnPBncPEQ81B+AR'),
                    'dtype': 'i2'},
              'yaxis': 'y2'},
             {'boxpoints': 'outliers',
              'line': {'color': 'black'},
              'marker': {'color': 'lightblue'},
              'name': 'Duration',
              'type': 'box',
              'x': array(['bad', 'good', 'bad', ..., 'good', 'bad', 'good'],
                         shape=(522,), dtype=object),
              'xaxis': 'x3',
              'y': {'bdata': ('MCoYJB4MMAwYDxgYBgoMBgwHPBIYEg' ... 'wYFQYbCw8YPBgGDx4VJA8qGBgkHi0t'),
                    'dtype': 'i1'},
              'yaxis': 'y3'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Age',
                                'x': 0.14444444444444446,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Credit amount',
                                'x': 0.5,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Duration',
                                'x': 0.8555555555555556,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'}],
               'margin': {'b': 40, 'l': 40, 'r': 40, 't': 60},
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Boxplots der Merkmale nach Risiko-Klasse'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 0.2888888888888889]},
               'xaxis2': {'anchor': 'y2', 'domain': [0.35555555555555557, 0.6444444444444445]},
               'xaxis3': {'anchor': 'y3', 'domain': [0.7111111111111111, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0]},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 1.0]},
               'yaxis3': {'anchor': 'x3', 'domain': [0.0, 1.0]}}
})

In den drei Boxplots sieht man den Vergleich der Merkmale Alter, Kreditbetrag und Laufzeit zwischen den Risiko-Klassen good (niedriges Risiko) und bad (hohes Risiko).
- Beim Alter erkennt man kaum Unterschiede: Sowohl bei „good“ als auch bei „bad“ liegen die meisten Personen zwischen 25 und 40 Jahren. Das Alter allein scheint also kein starker Einflussfaktor auf das Risiko zu sein.
- Beim Kreditbetrag zeigt sich, dass Kunden mit hohem Risiko (bad) im Durchschnitt höhere Kredite aufnehmen und auch deutlich mehr Ausreißer nach oben haben (teilweise über 15.000 €). Das spricht dafür, dass sehr hohe Kredite häufiger riskant sind.
- Besonders deutlich wird der Unterschied bei der Laufzeit (Duration): Kredite mit hohem Risiko haben im Mittel deutlich längere Laufzeiten. Während „good“-Kredite meist kürzer laufen, sieht man bei „bad“ viele Verträge mit 40 Monaten oder mehr.

Alter spielt für das Risiko nur eine geringe Rolle, während hohe Kreditbeträge und vor allem lange Laufzeiten stärker mit einem höheren Risiko verbunden sind. Nun wollen wir diese Unterschiede auch zahlenmäßig festhalten:

In [74]:
print(df.groupby("Risk")[["Age", "Credit amount", "Duration"]].mean())

            Age  Credit amount   Duration
Risk                                     
bad   34.147186    3881.090909  25.445887
good  35.477663    2800.594502  18.079038


Die Berechnung zeigt deutliche Unterschiede zwischen den Risiko-Klassen. Personen mit hohem Risiko (bad) haben im Durchschnitt etwas weniger Alter, dafür aber höhere Kreditbeträge (ca. 3.881 € gegenüber 2.801 €) und vor allem deutlich längere Laufzeiten (25 Monate gegenüber 18 Monaten). Das bestätigt, was wir zuvor in den Boxplots gesehen haben: lange Laufzeiten und hohe Kreditbeträge erhöhen das Risiko eines Kredits, während das Alter nur eine kleine Rolle spielt.
<br>
<br>
Nachdem wir die numerischen Merkmale bereits in Beziehung zum Risiko gesetzt haben, wollen wir nun die kategorialen Merkmale genauer untersuchen. Um die Unterschiede sichtbar zu machen, stellen wir für jedes dieser Merkmale ein Balkendiagramm dar. Dabei vergleichen wir die Verteilung der Kategorien getrennt nach Risiko-Klasse (good vs. bad). So erkennen wir zum Beispiel, ob bestimmte Wohnformen oder Zwecke häufiger mit riskanten Krediten verbunden sind:

In [75]:
# Subplot-Raster (3 Spalten pro Reihe):
cols_per_row = 3
rows = math.ceil(len(categorical_cols) / cols_per_row)

fig8 = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=categorical_cols,
    horizontal_spacing=0.08,
    vertical_spacing=0.15
)

for i, col in enumerate(categorical_cols):
    r = i // cols_per_row + 1
    c = i % cols_per_row + 1
    
    if col != "Risk":
        # Häufigkeiten nach Risk berechnen:
        value_counts = df.groupby([col, "Risk"]).size().reset_index(name="count")
        
        # Balken hinzufügen (good vs. bad):
        for risk_class in value_counts["Risk"].unique():
            subset = value_counts[value_counts["Risk"] == risk_class]
            fig8.add_trace(
                go.Bar(
                    x=subset[col].astype(str),
                    y=subset["count"],
                    name=risk_class,
                    marker_line_color="black",
                    marker_line_width=1
                ),
                row=r, col=c
            )
    else:
        # Nur einfache Verteilung von Risk selbst:
        value_counts = df[col].value_counts().reset_index()
        value_counts.columns = [col, "count"]
        
        fig8.add_trace(
            go.Bar(
                x=value_counts[col].astype(str),
                y=value_counts["count"],
                name=col,
                marker_line_color="black",
                marker_line_width=1
            ),
            row=r, col=c
        )

fig8.update_xaxes(showgrid=False, tickangle=45)
fig8.update_yaxes(showgrid=False)
fig8.update_layout(
    title="Verteilung der kategorialen Merkmale",
    barmode="group",
    height=350 * rows
)

fig8


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'bad',
              'type': 'bar',
              'x': array(['female', 'male'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'UgCVAA==', 'dtype': 'i2'},
              'yaxis': 'y'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'good',
              'type': 'bar',
              'x': array(['female', 'male'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'VgDNAA==', 'dtype': 'i2'},
              'yaxis': 'y'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'bad',
              'type': 'bar',
              'x': array(['free', 'own', 'rent'], dtype=object),
              'xaxis': 'x2',
              'y': {'bdata': 'JACLADgA', 'dtype': 'i2'},
              'yaxis': 'y2'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'good',
              'type': 'bar',
              'x': array(['free', 'own', 'rent'], dtype=object),
              'xaxis': 'x2',
              'y': {'bdata': 'HQDSADQA', 'dtype': 'i2'},
              'yaxis': 'y2'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'bad',
              'type': 'bar',
              'x': array(['little', 'moderate', 'quite rich', 'rich'], dtype=object),
              'xaxis': 'x3',
              'y': {'bdata': 'vwAeAAYABAA=', 'dtype': 'i2'},
              'yaxis': 'y3'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'good',
              'type': 'bar',
              'x': array(['little', 'moderate', 'quite rich', 'rich'], dtype=object),
              'xaxis': 'x3',
              'y': {'bdata': '3QAiABEAEwA=', 'dtype': 'i2'},
              'yaxis': 'y3'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'bad',
              'type': 'bar',
              'x': array(['little', 'moderate', 'rich'], dtype=object),
              'xaxis': 'x4',
              'y': {'bdata': 'eWIM', 'dtype': 'i1'},
              'yaxis': 'y4'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'good',
              'type': 'bar',
              'x': array(['little', 'moderate', 'rich'], dtype=object),
              'xaxis': 'x4',
              'y': {'bdata': 'fH4p', 'dtype': 'i1'},
              'yaxis': 'y4'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'bad',
              'type': 'bar',
              'x': array(['business', 'car', 'domestic appliances', 'education',
                          'furniture/equipment', 'radio/TV', 'repairs', 'vacation/others'],
                         dtype=object),
              'xaxis': 'x5',
              'y': {'bdata': 'GFADEi0zBgQ=', 'dtype': 'i1'},
              'yaxis': 'y5'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'good',
              'type': 'bar',
              'x': array(['business', 'car', 'domestic appliances', 'education',
                          'furniture/equipment', 'radio/TV', 'repairs', 'vacation/others'],
                         dtype=object),
              'xaxis': 'x5',
              'y': {'bdata': 'HV0DCj5RCAU=', 'dtype': 'i1'},
              'yaxis': 'y5'},
             {'marker': {'line': {'color': 'black', 'width': 1}},
              'name': 'Risk',
              'type': 'bar',
              'x': array(['good', 'bad'], dtype=object),
              'xaxis': 'x6',
              'y': {'bdata': 'IwHnAA==', 'dtype': 'i2'},
              'yaxis': 'y6'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Sex',
                                'x': 0.13999999999999999,
                                'xanchor': 'center',
                           

Die Übersicht zeigt, wie sich die Risiko-Klassen (good vs. bad) über die verschiedenen Kategorien verteilen:
- **Geschlecht (Sex):** Männer sind häufiger vertreten als Frauen. Ein klarer Unterschied zwischen good und bad ist aber nicht erkennbar – beide Geschlechter haben in etwa denselben Anteil an riskanten Krediten.
- **Wohnsituation (Housing):** Die meisten Kreditnehmer besitzen ein eigenes Haus (own). Mieter (rent) und Personen, die kostenlos wohnen (free), sind deutlich seltener. Auffällig ist, dass gerade bei Mietern der Anteil der bad-Kredite höher wirkt.
- **Sparguthaben (Saving accounts):** Viele Kunden haben nur geringe Ersparnisse (little). Hier treten auch besonders viele riskante Kredite auf. Bei höheren Guthaben-Klassen (moderate, quite rich, rich) sinkt die Zahl der riskanten Kredite deutlich.
- **Girokonto (Checking account):** Personen mit little oder moderate Guthaben auf dem Girokonto sind am häufigsten vertreten. Wer ein rich Konto hat, kommt deutlich seltener vor und scheint insgesamt etwas sicherer zu sein.
- **Kreditverwendungszweck (Purpose):** Am häufigsten werden Kredite für Auto oder Radio/TV aufgenommen. Bei Bildung (education) oder sonstigen Zwecken (vacation/others) fällt auf, dass der Anteil riskanter Kredite relativ hoch ist.
- **Gesamtverteilung (Risk):** Rund 56 % der Kredite gelten als good und 44 % als bad. Das hatten wir zuvor schon gesehen, wird hier aber noch einmal bestätigt.
<br>
<br>
Nachdem wir unsere Daten nun gründlich analysiert und die wichtigsten Merkmale besser verstanden haben, wollen wir den Datensatz für das Modellieren vorbereiten. Dazu wählen wir gezielt bestimmte Spalten aus, die wir später als Eingabevariablen (Features) verwenden möchten. Diese Variablen liefern die Informationen, auf deren Grundlage ein Modell lernen soll, wie sich Kreditrisiken unterscheiden.
<br>
<br>
Außerdem legen wir fest, welches Merkmal die Zielvariable (Target) ist. In unserem Fall ist das die Spalte Risk, also die Klassifikation in good oder bad.

In [76]:
features= ["Age", "Sex", "Job", "Housing", "Saving accounts", "Checking account", "Credit amount", "Duration", "Purpose"]
target = "Risk"
df_model = df[features + [target]].copy()
df_model

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,22,female,2,own,little,moderate,5951,48,radio/TV,bad
1,45,male,2,free,little,little,7882,42,furniture/equipment,good
2,53,male,2,free,little,little,4870,24,car,bad
3,35,male,3,rent,little,moderate,6948,36,car,good
4,28,male,3,own,little,moderate,5234,30,car,bad
...,...,...,...,...,...,...,...,...,...,...
517,48,male,1,own,little,moderate,1743,24,radio/TV,good
518,30,male,3,own,little,little,3959,36,furniture/equipment,good
519,40,male,3,own,little,little,3857,30,car,good
520,23,male,2,free,little,little,1845,45,radio/TV,bad


Viele unserer Merkmale sind kategorisch, zum Beispiel Geschlecht (male/female), Wohnsituation (own/rent/free) oder Saving accounts (little, moderate, rich). Solche Werte sind für uns Menschen leicht verständlich, aber ein Machine-Learning-Modell kann mit Texten nicht direkt arbeiten – es benötigt numerische Werte.
<br>
<br>
Deshalb müssen wir die Kategorien in Zahlen umwandeln. Dafür verwenden wir den sogenannten `LabelEncoder` aus der Bibliothek scikit-learn. Dieser ordnet jeder Kategorie eine Zahl zu (z. B. male = 1, female = 0).
<br>
<br>
Der Ablauf sieht so aus:
1. Wir suchen alle Spalten mit kategorischen Daten (außer der Zielvariable Risk).
2. Wir wenden für jede dieser Spalten einen LabelEncoder an, der die Texte in Zahlen übersetzt.
3. Wir speichern die Encoder mit joblib, sodass wir sie später im Modell oder in einer App wiederverwenden können (wichtig, um neue Daten genau gleich zu kodieren wie die Trainingsdaten).

Am Ende haben wir ein DataFrame, in dem alle Merkmale numerisch vorliegen und somit direkt für das Trainieren von Modellen genutzt werden können.
<br>
<br>
In unserem Projekt wollen wir Baumbasierte Modelle (z.B. Decision Trees, Random Forest, Extra Trees, XGBoost) verwenden. Diese Modelle können sehr gut mit Label Encoding umgehen, da sie Spalten nicht nach numerischer Größe, sondern nach Bedingungen (z.B. Sex == 0?) aufteilen. OHE wäre hier unnötig und würde den Datensatz nur größer und unübersichtlicher machen.
- Label Encoding reicht für baumartige Modelle (Tree-based Models).
- One-Hot-Encoding ist besser für lineare Modelle (z.B. lineare Regression, logistische Regression), weil dort Zahlenwerte direkt in Berechnungen eingehen.

In [77]:
from sklearn.preprocessing import LabelEncoder
import joblib

cat_cols = df_model.select_dtypes(include=["object"]).columns.drop("Risk")
le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    le_dict[col] = le
    joblib.dump(le, f"{col}_encoder.pkl")

df_model

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,22,0,2,1,0,1,5951,48,5,bad
1,45,1,2,0,0,0,7882,42,4,good
2,53,1,2,0,0,0,4870,24,1,bad
3,35,1,3,2,0,1,6948,36,1,good
4,28,1,3,1,0,1,5234,30,1,bad
...,...,...,...,...,...,...,...,...,...,...
517,48,1,1,1,0,1,1743,24,5,good
518,30,1,3,1,0,0,3959,36,4,good
519,40,1,3,1,0,0,3857,30,1,good
520,23,1,2,0,0,0,1845,45,5,bad


Bisher haben wir nur die Eingabevariablen (Features) umgewandelt, also alle Spalten, die Informationen über die Kreditnehmer enthalten. Nun müssen wir noch die Zielvariable (Risk) in Zahlen übersetzen, damit auch sie vom Modell verarbeitet werden kann:

In [78]:
le_target = LabelEncoder()
df_model[target] = le_target.fit_transform(df_model[target])

df_model[target]

0      0
1      1
2      0
3      1
4      0
      ..
517    1
518    1
519    1
520    0
521    1
Name: Risk, Length: 522, dtype: int64

Betrachten wir die Verteilung unserer Zielvariable:

In [79]:
print(df_model[target].value_counts())

Risk
1    291
0    231
Name: count, dtype: int64


Die Klasse 1 (das entspricht good) kommt häufiger vor als die Klasse 0 (bad).
Damit sehen wir, dass unser Datensatz eine leichte Ungleichverteilung hat – es gibt mehr sichere Kredite als riskante. Das ist einerseits positiv, weil wir genügend Daten aus beiden Gruppen haben. Andererseits müssen wir uns merken, dass das Modell später dazu neigen könnte, häufiger good vorherzusagen, weil diese Klasse öfter vorkommt.
<br>
<br>
Wir haben die Zielvariable Risk erfolgreich mit dem LabelEncoder in Zahlen übersetzt. Damit das später – zum Beispiel in einer Streamlit-Anwendung – genauso funktioniert, müssen wir den Encoder speichern. Mit dem folgenden Code speichern wir den Encoder als Datei (target_encoder.pkl). Diese Datei können wir später direkt wieder einlesen und in unserer Streamlit-App verwenden:

In [80]:
joblib.dump(le_target, "target_encoder.pkl")

['target_encoder.pkl']

Damit wir ein Machine-Learning-Modell trainieren und anschließend auch überprüfen können, ob es gute Vorhersagen macht, müssen wir unseren Datensatz in zwei Teile zerlegen:
- Trainingsdaten – mit diesen Daten "lernt" das Modell die Zusammenhänge zwischen Eingabevariablen (Features) und der Zielvariable (Risk).
- Testdaten – diese Daten sieht das Modell während des Trainings nicht. Sie dienen später dazu, die Qualität des Modells objektiv zu überprüfen.

Im Code verwenden wir dafür die `train_test_split`-Funktion aus scikit-learn:
- `X` enthält die Merkmale (Features).
- `y` ist unsere Zielvariable (Risk).
- Mit `test_size=0.2` reservieren wir 20 % der Daten für den Test.
- `stratify=y` sorgt dafür, dass die Klassenverteilung (good vs. bad) im Training und im Test gleich bleibt.
- `random_state=42` stellt sicher, dass wir bei jedem Durchlauf dieselbe Aufteilung bekommen – so sind die Ergebnisse reproduzierbar.

In [81]:
from sklearn.model_selection import train_test_split

X = df_model.drop(target, axis=1)
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"Geometrie von 'X': {X.shape}")
print(f"Geometrie von 'y': {y.shape}")

Geometrie von 'X': (522, 9)
Geometrie von 'y': (522,)


Jetzt, da unsere Daten sauber vorbereitet und in Trainings- und Testsets aufgeteilt sind, können wir mit dem eigentlichen Modelltraining beginnen. Dafür benötigen wir verschiedene Klassifikationsverfahren sowie Hilfsfunktionen zur Auswertung.
- `DecisionTreeClassifier`: Ein einzelner Entscheidungsbaum, der Regeln aus den Daten ableitet. Einfach, gut interpretierbar, aber anfällig für Überanpassung.
- `RandomForestClassifier`: Ein Ensemble vieler Entscheidungsbäume, das durch Abstimmung stabilere Ergebnisse liefert.
- `ExtraTreesClassifier`: Ähnlich wie der Random Forest, aber mit noch stärkerer Zufallsauswahl – oft schneller und manchmal genauer.
- `XGBClassifier (XGBoost)`: Ein modernes, leistungsstarkes Ensemble-Verfahren, das auf "Boosting" basiert und in vielen Wettbewerben sehr erfolgreich eingesetzt wird.

Zusätzlich brauchen wir:
- `accuracy_score`: um die Treffergenauigkeit (Accuracy) unserer Modelle zu berechnen.
- `GridSearchCV`: ein Werkzeug zur systematischen Suche nach den besten Hyperparametern, also den Einstellungen, mit denen das Modell am besten funktioniert.

In [82]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

Damit wir nun verschiedene Klassifikationsverfahren ausprobieren und fair vergleichen können, schreiben wir uns eine Hilfsfunktion. Die Funktion soll in der Lage sein, folgende aufgaben zu übernehmen:

**1. Hyperparameter-Suche:**
- Mit `GridSearchCV` testen wir automatisch verschiedene Kombinationen von Einstellungen (z.B. maximale Tiefe eines Baumes oder Anzahl der Bäume im Random Forest).
  - `cv=5` bedeutet, dass die Daten in 5 Falten aufgeteilt und getestet werden → das sorgt für eine robuste Bewertung.
  - `scoring="accuracy"` sagt, dass wir die Treffergenauigkeit als Maß verwenden.
  - `n_jobs=-1` nutzt alle verfügbaren Prozessorkerne, damit die Suche schneller geht.

**2. Bestes Modell auswählen:**
- Nach der Suche erhalten wir das Modell mit den besten Einstellungen (`best_estimator_`).

**3. Testdaten vorhersagen:**
- Mit diesem Modell sagen wir die Ergebnisse für unsere Testdaten voraus und berechnen die Accuracy.

**4. Alles zurückgeben:**
- Die Funktion liefert uns:
  - das beste Modell,
  - die erreichte Accuracy,
  - und die besten Hyperparameter.

So müssen wir nicht jedes Mal den gleichen Code schreiben, sondern können für jedes Modell einfach diese Funktion nutzen.

In [83]:
def train_model(model, param_grid, X_train, y_train, X_test, y_test):
    grid = GridSearchCV(model, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return best_model, acc, grid.best_params_

Wir starten mit einem Entscheidungsbaum als Basismodell. Mit `class_weight="balanced"` gleichen wir die ungleiche Klassenverteilung (good/bad) aus, indem Fehlklassifikationen der selteneren Klasse stärker gewichtet werden. Damit der Baum nicht zu einfach (unterfittet) oder zu komplex (überfittet) wird, probieren wir verschiedene Hyperparameter aus:

In [84]:
dt = DecisionTreeClassifier(random_state=42, class_weight="balanced")
dt_param_grid = {
    "max_depth": [3, 5, 7, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

Nachdem wir unser Entscheidungsbaum-Modell und das dazugehörige Parameter-Raster (`dt_param_grid`) definiert haben, wollen wir den Baum nun wirklich trainieren und die besten Einstellungen finden. Dazu verwenden wir unsere selbst geschriebene Funktion:

In [85]:
best_dt, acc_dt, params_dt = train_model(dt, dt_param_grid, X_train, y_train, X_test, y_test)

Nachdem wir den Entscheidungsbaum trainiert und mit GridSearch die besten Hyperparameter gefunden haben, wollen wir uns nun die Resultate anschauen:

In [86]:
print(f"Decision Tree Accuaracy: {acc_dt}")
print(f"Best parameters:\n{params_dt}")

Decision Tree Accuaracy: 0.6
Best parameters:
{'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}


Das Modell liegt in etwa bei 60% richtiger Vorhersagen. Das ist ein Anfang, aber noch nicht besonders hoch. Nachdem wir den Entscheidungsbaum getestet haben, wollen wir nun ein Random Forest Modell ausprobieren. Der Random Forest besteht nicht nur aus einem einzelnen Baum, sondern aus vielen Entscheidungsbäumen, die jeweils auf leicht veränderten Daten trainiert werden. Am Ende stimmen diese Bäume „ab“, und dadurch wird das Ergebnis meist stabiler und genauer:

In [87]:
rf = RandomForestClassifier(random_state=42, class_weight="balanced", n_jobs=-1)
rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 7, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}
best_rf, acc_rf, params_rf = train_model(rf, rf_param_grid, X_train, y_train, X_test, y_test)
print(f"Random Forest Accuaracy: {acc_rf}")
print(f"Best parameters:\n{params_rf}")

Random Forest Accuaracy: 0.638095238095238
Best parameters:
{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


Der Random Forest verbessert die Vorhersage im Vergleich zum einzelnen Entscheidungsbaum: Die Accuracy steigt auf ca. 0,63 (vorher ~0,60). Das ist typisch für Ensemble-Methoden: Viele Bäume mitteln individuelle Fehler und liefern stabilere Ergebnisse. Die besten Hyperparameter deuten auf ein eher konservatives Modell hin.
<br>
<br>
Nach Entscheidungsbaum und Random Forest probieren wir nun Extra Trees (Extremely Randomized Trees) aus – ebenfalls ein Ensemble vieler Bäume. Der entscheidende Unterschied:
- Während der Random Forest zufällig Merkmale auswählt und pro Merkmal den besten Split sucht,
- wählt Extra Trees zusätzlich die Split-Schwellen zufälliger.

Diese stärkere Zufälligkeit führt oft zu geringerer Varianz (weniger Overfitting) und kann auf Tabellendaten schneller und teilweise genauer sein.

In [88]:
et = ExtraTreesClassifier(random_state=42, class_weight="balanced", n_jobs=-1)
et_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 7, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}
best_et, acc_et, params_et = train_model(et, et_param_grid, X_train, y_train, X_test, y_test)
print(f"Extra trees Accuaracy: {acc_et}")
print(f"Best parameters:\n{params_et}")

Extra trees Accuaracy: 0.580952380952381
Best parameters:
{'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


Der Extra-Trees-Klassifikator erreicht mit einer Accuracy von ca. 0,63 praktisch das gleiche Niveau wie der Random Forest (und etwas besser als der einzelne Entscheidungsbaum). Das passt zur Idee von Extra Trees: Durch stärkere Zufälligkeit bei den Splits sinkt die Varianz des Modells, ohne dass die Genauigkeit leidet. Trees liefert robuste, wettbewerbsfähige Ergebnisse bei geringem Tuning-Aufwand. Für die weitere Bewertung sollte man neben der Accuracy auch Konfusionsmatrix, Precision/Recall/F1 (insbesondere für die „bad“-Klasse) und ROC-AUC betrachten.
<br>
<br>
Zum Abschluss wollen wir noch ein modernes Ensemble-Verfahren testen: den XGBoost-Classifier (Extreme Gradient Boosting).
XGBoost basiert auf Boosting, d. h. es werden viele schwache Entscheidungsbäume nacheinander trainiert. Jeder neue Baum versucht, die Fehler der vorherigen zu korrigieren. Dadurch entstehen oft sehr leistungsfähige Modelle, die in der Praxis und in vielen Kaggle-Wettbewerben erfolgreich eingesetzt werden.

Im Code achten wir auf zwei Dinge:
- Klassenungleichgewicht ausgleichen: Mit dem Parameter `scale_pos_weight` gewichten wir die seltener vorkommende Klasse stärker, damit das Modell nicht zu sehr die Mehrheitsklasse bevorzugt.
  - `(y_train == 0).sum()` zählt, wie viele „bad“-Fälle im Trainingsset sind.
  - `(y_train == 1).sum()` zählt, wie viele „good“-Fälle im Trainingsset sind.
  - Das Verhältnis dieser beiden Zahlen sagt uns, wie stark die Klassen unterschiedlich verteilt sind.
- `eval_metric="logloss"`: Diese Einstellung ist eine technische Vorgabe, damit XGBoost ohne Warnungen läuft und eine geeignete Verlustfunktion verwendet wird.

In [89]:
xgb = XGBClassifier(
    random_state=42,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),    
    eval_metric="logloss",       
    n_jobs=-1,                 
    tree_method="hist"      
)

xgb_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.7, 1.0],
    "colsample_bytree": [0.7, 1.0]
}

best_xgb, acc_xgb, params_xgb = train_model(
    xgb, xgb_param_grid, X_train, y_train, X_test, y_test
)

print(f"XGB Accuracy: {acc_xgb}")
print("Best parameters:", params_xgb)


XGB Accuracy: 0.6857142857142857
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.7}


Wir haben verschiedene Modelle getestet und ihre Genauigkeit verglichen:
- Entscheidungsbaum: ca. 60 % Accuracy, sehr einfaches Modell, aber noch nicht sehr genau.
- Random Forest / Extra Trees: ca. 63 % Accuracy, stabiler als der einzelne Baum, aber nur kleine Verbesserung.
- XGBoost: ca. 69 % Accuracy, bestes Ergebnis, klar besser als die anderen Modelle.

Da wir in der Praxis ein Modell haben wollen, das möglichst verlässliche Vorhersagen trifft, entscheiden wir uns für XGBoost. Für unsere Streamlit-Anwendung bedeutet das: Wir werden XGBoost als finales Modell exportieren und in die App einbauen. So können Nutzer dort später ihre Daten eingeben und eine Vorhersage über das Kreditrisiko bekommen – basierend auf dem leistungsstärksten Modell unserer Analyse.
<br>
<br>
Damit wir unser trainiertes Modell später in einer Anwendung (z.B. Streamlit) wiederverwenden können, müssen wir es dauerhaft abspeichern. Diese Datei enthält alle gelernten Regeln und Parameter des Modells. So können wir es später direkt laden, ohne erneut trainieren zu müssen – ein wichtiger Schritt für den praktischen Einsatz in unserer Streamlit-App.

In [90]:
joblib.dump(best_xgb, "xgb_credit_model.pkl")

['xgb_credit_model.pkl']

In [91]:
features_names = X_train.columns.to_list()
joblib.dump(features_names, "feature_names.pkl")

['feature_names.pkl']

## Aufgabenstellung

Ihr habt nun ein Machine-Learning-Modell trainiert (Extra Trees oder XGBoost), das anhand verschiedener Kundendaten das Kreditrisiko („good“ = geringes Risiko, „bad“ = hohes Risiko) einschätzen kann. Dieses Modell habt ihr bereits mit joblib als `.pkl`-Datei gespeichert.
<br>
<br>
Eure Aufgabe ist es jetzt, eine interaktive Web-App mit Streamlit zu entwickeln, die dieses Modell nutzt, um Vorhersagen für neue Kredit-Anfragen zu treffen.

**1. Funktionaler Umfang:**

Die App nimmt alle für das Modell nötigen Eingaben entgegen:
- Numerisch: Age, Credit amount, Duration
- Kategorisch: Sex, Job (0–3), Housing, Saving accounts, Checking account, Purpose
- Die App lädt das gespeicherte Modell (z. B. extra_trees_credit_model.pkl oder das beste XGBoost-Modell) und die Encoder für alle kategorialen Spalten sowie den Target-Encoder.
- Die App encodiert Nutzereingaben exakt so wie im Training (LabelEncoder aus den .pkl-Dateien).
- Die App erstellt eine Vorhersage (Risk: 0 = bad, 1 = good) und zeigt optional die Wahrscheinlichkeit (z. B. predict_proba) an.

**2. Benutzerschnittstelle (UI):**

Überschrift und kurze Erklärung, was die App macht (1–2 Sätze).

Eingabemasken:
- st.number_input für numerische Felder (mit sinnvollen Min-/Max-Werten und Defaults).
- st.selectbox für kategoriale Felder (Optionen müssen den Trainingskategorien entsprechen).

Buttons:
- „Vorhersage starten“ zur Berechnung.
- (Optional) „Eingaben zurücksetzen“.

Ausgabe:
- Deutlicher Status: st.success bei good, st.error bei bad.
- (Optional) Anzeige der Wahrscheinlichkeiten in Prozent.

Layout:
- Eingaben in der Sidebar oder in einem klar abgegrenzten Bereich; Ergebnis zentral.

**3. Datenvalidierung & Robustheit:**

Plausibilitätsgrenzen:
- Age: 18–80,
- Credit amount: ≥ 0 (realistisch: bis ~20 000),
- Duration: 1–72 Monate.
- Kategorische Optionen genau wie im Training (z. B. Saving accounts: little, moderate, quite rich, rich).
- Fehlermeldungen verständlich formulieren (z. B. wenn ein Encoder fehlt oder eine unbekannte Kategorie gewählt wird).

**4. Modell- und Encoder-Handling:**

- Modell und alle Encoder mit joblib.load laden:
- Feature-Encoder (z. B. Sex_encoder.pkl, Housing_encoder.pkl, …)
- target_encoder.pkl für die Rückübersetzung 0/1 → bad/good.
- Spaltenreihenfolge und Feature-Namen müssen exakt zur Trainingszeit passen.
- (Optional) Caching mit @st.cache_resource/@st.cache_data für schnelleres Laden.

**5. Nachvollziehbarkeit & Reproduzierbarkeit:**

- Fixe Random Seeds wurden im Training gesetzt (Info in der App erwähnen).
- In der App kurz notieren:
  - verwendetes Modell,
  - Metrik (Accuracy aus dem Test),
  - Datum/Version (z. B. „Modellstand vom …“).

Bringt gerne eure eigenen Ideen mit ein! Ziel ist es, dass am Ende nicht nur eine funktionierende Streamlit-App entsteht, sondern ein umfangreiches Projekt, das ihr z. B. auf GitHub in eurem Portfolio präsentieren könnt. Damit zeigt ihr zukünftigen Arbeitgebern oder in Bewerbungen ganz konkret, dass ihr in der Lage seid, komplette Machine-Learning-Pipelines inklusive Web-App umzusetzen.